In [37]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from requests.auth import HTTPBasicAuth
import time

In [2]:
client_id = 'HWJJKBTCHBRPBCMGZC9DZXFHQCCNXP'
client_secret = 'JHZWJWDSFQMMMQM4MRQPBNZTVNJPCHJFZCTJTKVZ'
scope = 'http://oauth.lexisnexis.com/all'
grant_type = 'client_credentials'
content_type = 'application/x-www-form-urlencoded'

In [3]:
# for details on authentication, see the docs at https://www.lexisnexis.com/lextalk/developers/ln-webservice-api/p/apidocs.aspx#introduction, section "Service to Service ID"
# also see https://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html#backend-application-flow 
auth = HTTPBasicAuth(client_id, client_secret)

client = BackendApplicationClient(client_id=client_id, scope=scope, grant_type=grant_type, token_type='Bearer', client_secret=client_secret, content_type=content_type)

oauth = OAuth2Session(client=client)

token = oauth.fetch_token(token_url='https://auth-api.lexisnexis.com/oauth/v2/token', auth=auth)

In [4]:
# get the token
token

{'access_token': 'NDI5MDEyNTUtODMzMy00ZjIzLThmYjUtNTU5MzdmNTczZTIw',
 'token_type': 'Bearer',
 'expires_in': 86400,
 'expires_at': 1718730587.957951}

In [34]:
# set up a request to the BatchNews endpoint; should only be used when returning > 500 documents 
base_url = 'https://services-api.lexisnexis.com/v1/'
search_endpoint = 'BatchNews'
search_string = 'microsoft w/5 (AI OR copilot)'
filter = 'Date ge 2024-01-01' # comparison operators: gt, ge, lt, le
results_per_page = 50 # max is 50 for BatchNews
url_params = {'$search': search_string, '$filter': filter, '$top': results_per_page}

In [35]:
# you need to pass the token in the headers by calling 'get' on the oauth object
r = oauth.get(base_url+search_endpoint, params=url_params, timeout=10)

In [36]:
r.json()

{'@odata.context': 'https://services-api.lexisnexis.com/v1/$metadata#BatchNews(ResultId,Source)',
 '@odata.count': 57826,
 'value': [{'ResultId': 'urn:contentItem:6C3J-3011-DXMP-K424-00000-00',
   'Source': {'Id': '', 'Name': 'TimesTech', 'ContentType': ''}},
  {'ResultId': 'urn:contentItem:6C3J-3011-DXMP-K241-00000-00',
   'Source': {'Id': '', 'Name': 'CXOtoday.com', 'ContentType': ''}},
  {'ResultId': 'urn:contentItem:6C34-TJ11-JB5M-W3FF-00000-00',
   'Source': {'Id': '', 'Name': 'DQ Channels India', 'ContentType': ''}},
  {'ResultId': 'urn:contentItem:6BKP-MC71-DXMP-K14K-00000-00',
   'Source': {'Id': '', 'Name': 'Taiwan Business News', 'ContentType': ''}},
  {'ResultId': 'urn:contentItem:6C34-TJ11-JB5M-W3N4-00000-00',
   'Source': {'Id': '', 'Name': 'Indian Express', 'ContentType': ''}},
  {'ResultId': 'urn:contentItem:6C81-T211-F11P-X4X0-00000-00',
   'Source': {'Id': '', 'Name': 'ITWeb', 'ContentType': ''}},
  {'ResultId': 'urn:contentItem:6C3H-YGT1-JDG9-Y4SM-00000-00',
   'Sourc

In [33]:
# get the document IDs
doc_ids = [doc['ResultId'] for doc in r.json()['value']]

In [38]:
# iterate through the pages of results and add the doc_ids to the list. Because of rate limits, you may need to add a sleep statement of 2-3 seconds, depending on the number of results
counter = 0
limit = 10  # Set your limit

while '@odata.nextLink' in r.json().keys() and counter < limit:
    r = oauth.get(r.json()['@odata.nextLink'])
    doc_ids.extend([doc['ResultId'] for doc in r.json()['value']])
    counter += 1  # Increment the counter while '@odata.nextLink' in r.json().keys():
    #time.sleep(2)

In [40]:
# iterate over the doc_ids and fetch the documents and save them to a file
for doc_id in doc_ids:
    docs_url = f"{base_url}Documents(DocumentId='{doc_id}',DocumentIdType='PGuid')/$value"
    doc = oauth.get(docs_url)
    with open(f'{doc_id}.xml', 'w') as f:
        f.write(doc.text)


KeyboardInterrupt: 

In [42]:
r.headers

{'Cache-Control': 'no-cache', 'Content-Type': 'application/json; odata.metadata=minimal', 'Date': 'Mon, 17 Jun 2024 19:50:38 GMT', 'Expires': '-1', 'OData-Version': '4.0', 'Pragma': 'no-cache', 'Server': 'auth_api.lexisnexis.com  3000', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers, Accept-Encoding', 'X-AspNet-Version': '4.0.30319', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'DENY', 'X-Powered-By': 'ASP.NET', 'X-RateLimit-Limit': '60/125/1500/12000', 'X-RateLimit-Remaining': '50/115/1485/11972', 'X-RateLimit-Reset': '1718653843/1718653893/1718655308/1718724658', 'X-XSS-Protection': '1; mode=block', 'Content-Length': '8097', 'Connection': 'keep-alive', 'X-RE-Ref': '1 1718653833019795', 'P3P': 'CP="IDC DSP LAW ADM DEV TAI PSA PSD IVA IVD CON HIS TEL OUR DEL SAM OTR IND OTC"'}